In [1]:
import pandas as pd
import numpy as np

import plotly.offline as py
import plotly.graph_objs as go

from statsmodels.tsa.arima_model import ARIMA

/usr/local/lib/python3.5/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



In [2]:
df = pd.read_csv('../../train_merged.csv', sep=',', header='infer')

In [3]:
ts = df[(df.Store == 1) & (df.Dept == 17)]
ts = ts[['Weekly_Sales', 'Date']]
ts = ts.set_index('Date')

In [4]:
std =  ts.rolling(window=52).std()
mean = ts.rolling(window=52).mean()
ewma = ts.ewm(halflife=12).mean()

In [5]:
data = [go.Scatter(x=ts.index, y=ts.Weekly_Sales, name='original'),
        go.Scatter(x=std.index, y=std.Weekly_Sales, name='std'),
        go.Scatter(x=mean.index, y=mean.Weekly_Sales, name='mean'),
        go.Scatter(x=ewma.index, y=ewma.Weekly_Sales, name='ewma')]
# py.plot(data)

'file:///home/kanoonsantikul/Documents/walmart-sell-predict/src/time-series/temp-plot.html'

In [6]:
ts_diff = ts - mean
ts_diff.dropna(inplace=True)
# py.plot([go.Scatter(x=ts_diff.index, y=ts_diff.Weekly_Sales, name='diff'),
#         go.Scatter(x=ts.index, y=ts.Weekly_Sales, name='original')])

'file:///home/kanoonsantikul/Documents/walmart-sell-predict/src/time-series/temp-plot.html'

In [7]:
ts_diff = ts - ewma
ts_diff.dropna(inplace=True)
# py.plot([go.Scatter(x=ts_diff.index, y=ts_diff.Weekly_Sales, name='diff'),
#         go.Scatter(x=ts.index, y=ts.Weekly_Sales, name='original')])

'file:///home/kanoonsantikul/Documents/walmart-sell-predict/src/time-series/temp-plot.html'

In [ ]:
model = ARIMA(ts_diff, order=(0, 1, 2))  
results_ARIMA = model.fit()

yhat = results_ARIMA.fittedvalues
yhat *= -1

# py.plot([go.Scatter(x=yhat.index, y=yhat.values, name='ARIMA'),
#     go.Scatter(x=ts_diff.index, y=ts_diff.Weekly_Sales, name='original')])   

In [94]:
from datetime import datetime
import math

end_date = datetime.strptime(ts.index[-1], '%Y-%m-%d')

def eval_weeknum(x):
    return math.ceil((x - end_date).days / 7)

test = pd.read_csv('../../data/sampleSubmission.csv', sep='_')
test['Date'] = pd.to_datetime(test['Date'])
test['WeekNum'] = test['Date'].apply(eval_weeknum)
test

,Store,Dept,Date,WeekNum
0,1,1,2012-11-02,1
1,1,1,2012-11-09,2
2,1,1,2012-11-16,3
3,1,1,2012-11-23,4
4,1,1,2012-11-30,5
5,1,1,2012-12-07,6
6,1,1,2012-12-14,7
7,1,1,2012-12-21,8
8,1,1,2012-12-28,9
9,1,1,2013-01-04,10


In [92]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return np.array(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]


def predict (store, dept, weeknum):
    ts = df[(df.Store == store) & (df.Dept == dept)]
    ts = ts[['Weekly_Sales', 'Date']]
    ts = ts.set_index('Date')
    
    series = ts
    # seasonal difference
    X = series.values
    weeks_in_year = 52
    differenced = difference(X, weeks_in_year)
    # fit model
    model = ARIMA(differenced, order=(0,1,2))
    model_fit = model.fit(disp=0)
    # multi-step out-of-sample forecast

    start_index = len(differenced)
    end_index = start_index + weeknum - 1

    forecast = model_fit.predict(start=start_index, end=end_index)

    # invert the differenced forecast to something usable
    history = [x for x in X]
    for yhat in forecast:
        inverted = inverse_difference(history, yhat, weeks_in_year)
        history.append(inverted)

    inverse = []
    for yhat in history:
        inverse.append(yhat[0])
    py.plot([go.Scatter(y=inverse), go.Scatter(y=ts.Weekly_Sales)])

    print(len(forecast), len(differenced))
    return inverse[-1]

In [93]:
predict(1,17,1)

1 91


7524.8190234181957